In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from clust_huge_amp import *



## here we're running with the likelihood only 

In [2]:
%load_ext autoreload
%autoreload 2



%autoreload 2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.spatial.distance import pdist, squareform
from scipy.special import expit
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering  # Add this import

def load_model_essentials(base_path='/Users/sarahurbut/Library/CloudStorage/Dropbox/data_for_running/'):
    """
    Load all essential components
    """
    print("Loading components...")
    
    # Load large matrices
    Y = torch.load(base_path + 'Y_tensor.pt')
    E = torch.load(base_path + 'E_matrix.pt')
    G = torch.load(base_path + 'G_matrix.pt')
    
    # Load other components
    essentials = torch.load(base_path + 'model_essentials.pt')
    
    print("Loaded all components successfully!")
    
    return Y, E, G, essentials

# Load and initialize model:
Y, E, G, essentials = load_model_essentials()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading components...


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_62642/432458031.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Y = torch.load(base_path + 'Y_tensor.pt')
/va

Loaded all components successfully!


In [3]:

# Subset the data
Y_100k, E_100k, G_100k, indices = subset_data(Y, E, G, start_index=0, end_index=10000)

torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Initialize model with subsetted data

del Y

In [4]:
# When initializing the model:
original_G = G_100k # Store the original G - proper tensor copy

# Now in your batch run, load and verify:
initial_psi = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/data_for_running/initial_psi_400k.pt')
initial_clusters = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/data_for_running/initial_clusters_400k.pt')



/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_62642/2496632321.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  initial_psi = torch.load('/Users/sarahurbut/L

In [5]:
print("Initial psi stats:")
print(f"Shape: {initial_psi.shape}")
print(f"Range: [{initial_psi.min():.2f}, {initial_psi.max():.2f}]")
print(f"Number of positive values: {(initial_psi > 0).sum().item()}")



Initial psi stats:
Shape: torch.Size([20, 348])
Range: [-2.04, 1.33]
Number of positive values: 348


In [6]:
# Load references (signatures only, no healthy)
refs = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/data_for_running/reference_trajectories.pt')
signature_refs = refs['signature_refs']
# When initializing the model:


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_62642/3625861070.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  refs = torch.load('/Users/sarahurbut/Library/

In [7]:

torch.manual_seed(7)
np.random.seed(4)
# Create model without healthy reference
model = AladynSurvivalFixedKernelsAvgLoss_clust_logitInit_psitest(
    N=Y_100k.shape[0], 
    D=Y_100k.shape[1], 
    T=Y_100k.shape[2], 
    K=20,
    P=G_100k.shape[1],
    init_sd_scaler=1e-1,
    G=G_100k, 
    Y=Y_100k,
    genetic_scale=1,
    W=0.0001,
    R=0,
    prevalence_t=essentials['prevalence_t'],
    signature_references=signature_refs,  # Only pass signature refs
    healthy_reference=True,  # Explicitly set to None
    disease_names=essentials['disease_names']
)

torch.manual_seed(0)
np.random.seed(0)
# Initialize with psi and clusters
model.initialize_params(true_psi=initial_psi)
model.clusters = initial_clusters
# Verify clusters match
clusters_match = np.array_equal(initial_clusters, model.clusters)
print(f"\nClusters match exactly: {clusters_match}")

/Users/sarahurbut/aladynoulli2/pyScripts/clust_huge_amp.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.signature_refs = torch.tensor(signature_references, dtype=torch.float32)
/Users/sarahurbut/aladynoulli2/pyScripts/clust_huge_amp.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.G = torch.tensor(G, dtype=torch.float32)
/Users/sarahurbut/aladynoulli2/pyScripts/clust_huge_amp.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.G = torch.tensor(G_scaled, dtype=torch.float32)
/Users/sarahurbut/aladynoulli2/pyScripts


Cluster Sizes:
Cluster 0: 14 diseases
Cluster 1: 7 diseases
Cluster 2: 21 diseases
Cluster 3: 15 diseases
Cluster 4: 17 diseases
Cluster 5: 16 diseases
Cluster 6: 57 diseases
Cluster 7: 18 diseases
Cluster 8: 13 diseases
Cluster 9: 11 diseases
Cluster 10: 18 diseases
Cluster 11: 12 diseases
Cluster 12: 26 diseases
Cluster 13: 7 diseases
Cluster 14: 9 diseases
Cluster 15: 8 diseases
Cluster 16: 7 diseases
Cluster 17: 11 diseases
Cluster 18: 6 diseases
Cluster 19: 55 diseases

Calculating gamma for k=0:
Number of diseases in cluster: 14
Base value (first 5): tensor([-13.8155, -13.8155, -13.1095, -12.4036, -12.4036])
Base value centered (first 5): tensor([-0.3723, -0.3723,  0.3336,  1.0396,  1.0396])
Base value centered mean: 6.57081614008348e-07
Gamma init for k=0 (first 5): tensor([ 0.0009,  0.0069,  0.0118,  0.0153, -0.0106])

Calculating gamma for k=1:
Number of diseases in cluster: 7
Base value (first 5): tensor([-13.8155, -13.8155, -13.8155, -13.8155, -13.8155])
Base value centered

In [8]:
print(model.K_total)
print(model.K)
print(model.lrtpen)
print(model.gpweight)
print(model.lambda_amplitude)

21
20
0
0.0001
1


In [29]:
## do the cluster assignemtns still map?
checkpoint_path = '/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_0_10000/model.pt'
checkpoint=torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
final_psi= model.psi.detach().clone()
# Get the signature with maximum psi for each disease
max_psi_indices = torch.argmax(final_psi, dim=0)  # Shape: [D]

# Compare with cluster assignments
matches = 0
total = len(model.clusters)

print("Comparing max psi signatures with cluster assignments:")
for k in range(model.K):
    # Find diseases assigned to cluster k
    cluster_diseases = np.where(model.clusters == k)[0]
    # Find diseases where k has maximum psi
    max_psi_diseases = np.where(max_psi_indices == k)[0]
    
    overlap = set(cluster_diseases) & set(max_psi_diseases)
    
    print(f"\nCluster {k}:")
    print(f"Number of diseases in cluster: {len(cluster_diseases)}")
    print(f"Number of diseases where this signature has max psi: {len(max_psi_diseases)}")
    print(f"Overlap: {len(overlap)}")
    
    matches += len(overlap)

print(f"\nOverall agreement: {matches}/{total} diseases ({matches/total*100:.1f}%)")

Comparing max psi signatures with cluster assignments:

Cluster 0:
Number of diseases in cluster: 16
Number of diseases where this signature has max psi: 16
Overlap: 16

Cluster 1:
Number of diseases in cluster: 21
Number of diseases where this signature has max psi: 21
Overlap: 21

Cluster 2:
Number of diseases in cluster: 15
Number of diseases where this signature has max psi: 15
Overlap: 15

Cluster 3:
Number of diseases in cluster: 82
Number of diseases where this signature has max psi: 82
Overlap: 82

Cluster 4:
Number of diseases in cluster: 5
Number of diseases where this signature has max psi: 5
Overlap: 5

Cluster 5:
Number of diseases in cluster: 7
Number of diseases where this signature has max psi: 7
Overlap: 7

Cluster 6:
Number of diseases in cluster: 8
Number of diseases where this signature has max psi: 8
Overlap: 8

Cluster 7:
Number of diseases in cluster: 22
Number of diseases where this signature has max psi: 22
Overlap: 22

Cluster 8:
Number of diseases in cluster:

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_62642/808960485.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load(checkpoint_path)


In [30]:
model.psi[:,112]

tensor([-2.1480, -2.0548, -2.0247, -2.2426, -1.9152, -1.0853, -2.0680, -2.0664,
        -2.0365, -2.1647, -2.2525, -2.0500, -2.0630, -1.9987, -1.9478, -1.9354,
        -2.3395, -2.1510, -1.9853, -2.0770, -4.9738],
       grad_fn=<SelectBackward0>)

In [11]:
clusters_match = np.array_equal(initial_clusters, model.clusters)
print(f"\nClusters match exactly: {clusters_match}")


Clusters match exactly: True


In [12]:
model.clusters[17]

np.int32(19)

In [13]:
model.gpweight

0.0001

In [14]:
# Get μ_dt from checkpoint and predictions
mu_dt = model.prevalence_t  # Shape: (348, 52)



In [15]:
def plot_signature_top_diseases_centered(model, disease_names, n_top=10):
    """
    Show top diseases for each signature, centered relative to prevalence
    """
    # Get phi and prevalence
    phi = model.phi.detach().numpy()  # Shape: (K, D, T)
    prevalence_logit = model.logit_prev_t.detach().numpy()  # Shape: (D, T)
    
    # Center phi relative to prevalence
    phi_centered = np.zeros_like(phi)
    for k in range(phi.shape[0]):
        for d in range(phi.shape[1]):
            phi_centered[k, d, :] = phi[k, d, :] - prevalence_logit[d, :]
    
    # Average over time
    phi_avg = phi_centered.mean(axis=2)  # Shape: (K, D)
    
    # For each signature, get top diseases
    for k in range(phi_avg.shape[0]):
        scores = phi_avg[k, :]
        top_indices = np.argsort(scores)[-n_top:][::-1]
        
        print(f"\nTop {n_top} diseases in Signature {k} (relative to baseline):")
        for idx in top_indices:
            avg_effect = scores[idx]
            temporal_std = np.std(phi_centered[k, idx, :])
            # Convert to odds ratio for interpretability
            odds_ratio = np.exp(avg_effect)
            print(f"{disease_names[idx]}: effect={avg_effect:.3f} (OR={odds_ratio:.2f}), std={temporal_std:.3f}")

# Run visualization
plot_signature_top_diseases_centered(model, essentials['disease_names'])


Top 10 diseases in Signature 0 (relative to baseline):
Atrial fibrillation and flutter: effect=1.591 (OR=4.91), std=0.603
Heart failure NOS: effect=1.470 (OR=4.35), std=0.328
Paroxysmal ventricular tachycardia: effect=1.273 (OR=3.57), std=0.111
Aortic valve disease: effect=1.270 (OR=3.56), std=0.159
Congestive heart failure (CHF) NOS: effect=1.260 (OR=3.52), std=0.306
Cardiomegaly: effect=1.248 (OR=3.48), std=0.341
Pleurisy; pleural effusion: effect=1.236 (OR=3.44), std=0.407
Mitral valve disease: effect=1.229 (OR=3.42), std=0.380
Congenital anomalies of great vessels: effect=1.182 (OR=3.26), std=0.244
Pericarditis: effect=1.169 (OR=3.22), std=0.072

Top 10 diseases in Signature 1 (relative to baseline):
Hallux valgus (Bunion): effect=1.289 (OR=3.63), std=0.138
Enthesopathy: effect=1.273 (OR=3.57), std=0.230
Peripheral enthesopathies and allied syndromes: effect=1.254 (OR=3.50), std=0.110
Pain in joint: effect=1.231 (OR=3.43), std=0.160
Internal derangement of knee: effect=1.226 (OR=3

In [16]:
model2=model
checkpoint_path2='/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_10000_20000/model.pt'
checkpoint2=torch.load(checkpoint_path2)
model2.load_state_dict(checkpoint2['model_state_dict'])
plot_signature_top_diseases_centered(model2, essentials['disease_names'])


Top 10 diseases in Signature 0 (relative to baseline):
Atrial fibrillation and flutter: effect=1.629 (OR=5.10), std=0.598
Heart failure NOS: effect=1.535 (OR=4.64), std=0.361
Paroxysmal ventricular tachycardia: effect=1.299 (OR=3.66), std=0.131
Aortic valve disease: effect=1.298 (OR=3.66), std=0.162
Congestive heart failure (CHF) NOS: effect=1.274 (OR=3.58), std=0.277
Cardiomegaly: effect=1.264 (OR=3.54), std=0.361
Left bundle branch block: effect=1.209 (OR=3.35), std=0.252
Mitral valve disease: effect=1.189 (OR=3.28), std=0.336
Congenital anomalies of great vessels: effect=1.148 (OR=3.15), std=0.237
Pericarditis: effect=1.144 (OR=3.14), std=0.067

Top 10 diseases in Signature 1 (relative to baseline):
Hallux valgus (Bunion): effect=1.267 (OR=3.55), std=0.130
Enthesopathy: effect=1.254 (OR=3.50), std=0.244
Arthropathy NOS: effect=1.238 (OR=3.45), std=0.322
Other peripheral nerve disorders: effect=1.232 (OR=3.43), std=0.190
Internal derangement of knee: effect=1.217 (OR=3.38), std=0.25

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_62642/1160586278.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint2=torch.load(checkpoint_path2)


In [ ]:
model3=model
checkpoint_path3='/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_20000_30000/model.pt'
checkpoint3=torch.load(checkpoint_path3)
model3.load_state_dict(checkpoint3['model_state_dict'])
plot_signature_top_diseases_centered(model3, essentials['disease_names'])


Top 10 diseases in Signature 0 (relative to baseline):
Atrial fibrillation and flutter: effect=1.629 (OR=5.10), std=0.598
Heart failure NOS: effect=1.535 (OR=4.64), std=0.361
Paroxysmal ventricular tachycardia: effect=1.299 (OR=3.66), std=0.131
Aortic valve disease: effect=1.298 (OR=3.66), std=0.162
Congestive heart failure (CHF) NOS: effect=1.274 (OR=3.58), std=0.277
Cardiomegaly: effect=1.264 (OR=3.54), std=0.361
Left bundle branch block: effect=1.209 (OR=3.35), std=0.252
Mitral valve disease: effect=1.189 (OR=3.28), std=0.336
Congenital anomalies of great vessels: effect=1.148 (OR=3.15), std=0.237
Pericarditis: effect=1.144 (OR=3.14), std=0.067

Top 10 diseases in Signature 1 (relative to baseline):
Hallux valgus (Bunion): effect=1.267 (OR=3.55), std=0.130
Enthesopathy: effect=1.254 (OR=3.50), std=0.244
Arthropathy NOS: effect=1.238 (OR=3.45), std=0.322
Other peripheral nerve disorders: effect=1.232 (OR=3.43), std=0.190
Internal derangement of knee: effect=1.217 (OR=3.38), std=0.25

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_62642/898760373.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint3=torch.load(checkpoint_path2)


<All keys matched successfully>

In [36]:
checkpoint_path

'/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_0_10000/model.pt'

In [37]:
checkpoint=torch.load(checkpoint_path)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_62642/2474169178.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load(checkpoint_path)


In [44]:
checkpoint=torch.load(checkpoint_path)
print(checkpoint['model_state_dict']['psi'][5,110:115])

checkpoint_path2='/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_10000_20000/model.pt'
checkpoint2=torch.load(checkpoint_path2)
print(checkpoint2['model_state_dict']['psi'][5,110:115])


checkpoint_path3='/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_20000_30000/model.pt'
checkpoint3=torch.load(checkpoint_path3)
print(checkpoint3['model_state_dict']['psi'][5,110:115])


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_62642/1593820600.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load(checkpoint_path)
/var/f

tensor([-2.0441,  1.6957, -1.0853, -0.8561,  3.0420])
tensor([-2.0427, -0.7331, -1.4377, -1.3729, -0.3947])
tensor([-2.0388, -0.5180, -0.0890, -1.1835,  0.8638])


In [ ]:
def plot_signature_top_diseases_centered(model, disease_names, n_top=10):
    """
    Show top diseases for each signature, centered relative to prevalence
    """
    # Get phi and prevalence
    phi = model.phi.detach().numpy()  # Shape: (K, D, T)
    prevalence_logit = model.logit_prev_t.detach().numpy()  # Shape: (D, T)
    
    # Center phi relative to prevalence
    phi_centered = np.zeros_like(phi)
    for k in range(phi.shape[0]):
        for d in range(phi.shape[1]):
            phi_centered[k, d, :] = phi[k, d, :] - prevalence_logit[d, :]
    
    # Average over time
    phi_avg = phi_centered.mean(axis=2)  # Shape: (K, D)
    
    # For each signature, get top diseases
    for k in range(phi_avg.shape[0]):
        scores = phi_avg[k, :]
        top_indices = np.argsort(scores)[-n_top:][::-1]
        
        print(f"\nTop {n_top} diseases in Signature {k} (relative to baseline):")
        for idx in top_indices:
            avg_effect = scores[idx]
            temporal_std = np.std(phi_centered[k, idx, :])
            # Convert to odds ratio for interpretability
            odds_ratio = np.exp(avg_effect)
            print(f"{disease_names[idx]}: effect={avg_effect:.3f} (OR={odds_ratio:.2f}), std={temporal_std:.3f}")

# Run visualization
plot_signature_top_diseases_centered(model, essentials['disease_names'])


Top 10 diseases in Signature 0 (relative to baseline):
Atrial fibrillation and flutter: effect=1.591 (OR=4.91), std=0.603
Heart failure NOS: effect=1.470 (OR=4.35), std=0.328
Paroxysmal ventricular tachycardia: effect=1.273 (OR=3.57), std=0.111
Aortic valve disease: effect=1.270 (OR=3.56), std=0.159
Congestive heart failure (CHF) NOS: effect=1.260 (OR=3.52), std=0.306
Cardiomegaly: effect=1.248 (OR=3.48), std=0.341
Pleurisy; pleural effusion: effect=1.236 (OR=3.44), std=0.407
Mitral valve disease: effect=1.229 (OR=3.42), std=0.380
Congenital anomalies of great vessels: effect=1.182 (OR=3.26), std=0.244
Pericarditis: effect=1.169 (OR=3.22), std=0.072

Top 10 diseases in Signature 1 (relative to baseline):
Hallux valgus (Bunion): effect=1.289 (OR=3.63), std=0.138
Enthesopathy: effect=1.273 (OR=3.57), std=0.230
Peripheral enthesopathies and allied syndromes: effect=1.254 (OR=3.50), std=0.110
Pain in joint: effect=1.231 (OR=3.43), std=0.160
Internal derangement of knee: effect=1.226 (OR=3

In [39]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [40]:
print(model3.psi[5,110:115])
print(model2.psi[5,110:115])
print(model.psi[5,110:115])

tensor([-2.0441,  1.6957, -1.0853, -0.8561,  3.0420], grad_fn=<SliceBackward0>)
tensor([-2.0441,  1.6957, -1.0853, -0.8561,  3.0420], grad_fn=<SliceBackward0>)
tensor([-2.0441,  1.6957, -1.0853, -0.8561,  3.0420], grad_fn=<SliceBackward0>)


In [41]:
initial_psi[5,110:115]

tensor([-2.0090,  0.8195,  0.8679,  0.8406,  1.1785])

In [46]:
import pandas as pd

def get_top_diseases_for_signatures(model, disease_names, n_top=4):
    """
    Calculates and returns the top N diseases for each signature based on centered phi.
    Returns a dictionary mapping signature index to a list of top disease info.
    """
    # Get phi and prevalence
    phi = model.phi.detach().numpy()
    prevalence_logit = model.logit_prev_t.detach().numpy()
    
    # Center phi relative to prevalence
    phi_centered = np.zeros_like(phi)
    for k in range(phi.shape[0]):
        for d in range(phi.shape[1]):
            phi_centered[k, d, :] = phi[k, d, :] - prevalence_logit[d, :]
            
    # Average over time
    phi_avg = phi_centered.mean(axis=2)
    
    signature_results = {}
    # For each disease signature, get top diseases
    for k in range(model.K): # model.K is the number of disease signatures
        scores = phi_avg[k, :]
        top_indices = np.argsort(scores)[-n_top:][::-1]
        
        top_diseases = []
        for idx in top_indices:
            avg_effect = scores[idx]
            odds_ratio = np.exp(avg_effect)
            top_diseases.append({
                "disease": disease_names[idx],
                "effect": avg_effect,
                "OR": odds_ratio
            })
        signature_results[k] = top_diseases
        
    return signature_results

# --- Analysis over multiple checkpoints ---

# Define checkpoint paths
checkpoint_paths = [
    '/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_0_10000/model.pt',
    '/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_10000_20000/model.pt',
    '/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_20000_30000/model.pt'
]

# Store results from all checkpoints
all_results = []

# Loop through each checkpoint
for i, path in enumerate(checkpoint_paths):
    print(f"Processing checkpoint {i+1} ({path.split('/')[-2]})...")
    # Add weights_only=True to avoid security warnings and because we only need the state dict
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Get top diseases for this model state
    top_diseases = get_top_diseases_for_signatures(model, essentials['disease_names'], n_top=4)
    all_results.append(top_diseases)

print("\nProcessing complete.")


# --- Display consolidated results ---

# Consolidate results for easier display
consolidated_results = {}
for k in range(model.K): # For each signature
    consolidated_results[k] = []
    for i in range(len(all_results)): # For each checkpoint
        # Store just the disease name for the summary table
        checkpoint_diseases = [d['disease'] for d in all_results[i][k]]
        consolidated_results[k].append(checkpoint_diseases)

# Display as a DataFrame for clarity
for k in range(model.K):
    print(f"--- Signature {k} ---")
    df = pd.DataFrame(
        consolidated_results[k],
        index=[f'Run {i+1}' for i in range(len(checkpoint_paths))],
        columns=[f'Top {j+1}' for j in range(4)]
    )
    print(df)
    print("\n")

Processing checkpoint 1 (output_0_10000)...
Processing checkpoint 2 (output_10000_20000)...


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_62642/3374945265.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


Processing checkpoint 3 (output_20000_30000)...

Processing complete.
--- Signature 0 ---
                                 Top 1              Top 2  \
Run 1  Atrial fibrillation and flutter  Heart failure NOS   
Run 2  Atrial fibrillation and flutter  Heart failure NOS   
Run 3  Atrial fibrillation and flutter  Heart failure NOS   

                                    Top 3                     Top 4  
Run 1  Paroxysmal ventricular tachycardia      Aortic valve disease  
Run 2  Paroxysmal ventricular tachycardia      Aortic valve disease  
Run 3                        Cardiomegaly  Left bundle branch block  


--- Signature 1 ---
                        Top 1         Top 2  \
Run 1  Hallux valgus (Bunion)  Enthesopathy   
Run 2  Hallux valgus (Bunion)  Enthesopathy   
Run 3  Hallux valgus (Bunion)  Enthesopathy   

                                                Top 3  \
Run 1  Peripheral enthesopathies and allied syndromes   
Run 2                                 Arthropathy NOS   
Run

In [49]:
import pandas as pd
from collections import Counter
import numpy as np

def escape_latex(text):
    """A simple function to escape common LaTeX special characters."""
    return text.replace('&', '\\&').replace('%', '\\%').replace('$', '\\$').replace('#', '\\#').replace('_', '\\_')

def get_top_diseases_with_ci(model, disease_names, n_top=4):
    """
    Calculates and returns top diseases with ORs and 95% CIs.
    CI is based on the temporal standard deviation of the centered phi.
    """
    # Get phi, prevalence, and number of time points (T)
    phi = model.phi.detach().numpy()
    prevalence_logit = model.logit_prev_t.detach().numpy()
    T = phi.shape[2]
    
    # Center phi relative to prevalence
    phi_centered = np.zeros_like(phi)
    for k in range(phi.shape[0]):
        for d in range(phi.shape[1]):
            phi_centered[k, d, :] = phi[k, d, :] - prevalence_logit[d, :]
            
    phi_avg = phi_centered.mean(axis=2)
    
    signature_results = {}
    for k in range(model.K): # model.K is the number of disease signatures
        scores = phi_avg[k, :]
        top_indices = np.argsort(scores)[-n_top:][::-1]
        
        top_diseases_info = []
        for idx in top_indices:
            avg_effect = scores[idx]
            temporal_std = np.std(phi_centered[k, idx, :])
            
            # Calculate 95% CI for the effect (log-odds)
            z_score = 1.96
            std_error_mean = temporal_std / np.sqrt(T)
            effect_ci_lower = avg_effect - z_score * std_error_mean
            effect_ci_upper = avg_effect + z_score * std_error_mean
            
            # Convert to OR and its CI by exponentiating
            top_diseases_info.append({
                "disease": disease_names[idx],
                "OR": np.exp(avg_effect),
                "OR_CI_lower": np.exp(effect_ci_lower),
                "OR_CI_upper": np.exp(effect_ci_upper)
            })
        signature_results[k] = top_diseases_info
        
    return signature_results

# --- Analysis over multiple checkpoints ---

# Define checkpoint paths (make sure 'model' is loaded from a checkpoint first)
checkpoint_paths = [
    '/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_0_10000/model.pt',
    '/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_10000_20000/model.pt',
    '/Users/sarahurbut/Library/CloudStorage/Dropbox/resultshighamp/results/output_20000_30000/model.pt'
]

# Store detailed results from all checkpoints
all_results_with_ci = []

for i, path in enumerate(checkpoint_paths):
    print(f"Processing checkpoint {i+1}...")
    checkpoint = torch.load(path, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    top_diseases = get_top_diseases_with_ci(model, essentials['disease_names'], n_top=4)
    all_results_with_ci.append(top_diseases)

print("\\nProcessing complete.")


# --- Consolidate and Generate LaTeX Table ---

latex_table_rows = []
for k in range(model.K):
    disease_counter = Counter()
    or_accumulator = {}

    for run_results in all_results_with_ci:
        for disease_info in run_results[k]:
            name = disease_info['disease']
            disease_counter[name] += 1
            if name not in or_accumulator:
                or_accumulator[name] = []
            or_accumulator[name].append({
                'or': disease_info['OR'],
                'lower': disease_info['OR_CI_lower'],
                'upper': disease_info['OR_CI_upper']
            })

    top_3_consistent = disease_counter.most_common(3)
    
    for disease_name, _ in top_3_consistent:
        avg_or = np.mean([d['or'] for d in or_accumulator[disease_name]])
        avg_lower = np.mean([d['lower'] for d in or_accumulator[disease_name]])
        avg_upper = np.mean([d['upper'] for d in or_accumulator[disease_name]])
        
        latex_table_rows.append({
            'signature': k,
            'disease': disease_name,
            'or_ci_str': f"{avg_or:.2f} ({avg_lower:.2f} - {avg_upper:.2f})"
        })

# Generate the LaTeX string
latex_string = "\\begin{table}[h!]\n"
latex_string += "\\centering\n"
latex_string += "\\caption{Top 3 Most Consistent Diseases per Signature with Average Odds Ratio and 95\\% CI}\n"
latex_string += "\\label{tab:signature_summary_ci}\n"
latex_string += "\\begin{tabular}{c p{9cm} c}\n"
latex_string += "\\toprule\n"
latex_string += "\\textbf{Signature} & \\textbf{Top Consistently Associated Disease} & \\textbf{Avg. OR (95\\% CI)} \\\\\n"
latex_string += "\\midrule\n"

current_sig = -1
for row in latex_table_rows:
    sig_num = row['signature']
    if sig_num != current_sig and current_sig != -1:
        latex_string += "\\midrule\n"
    
    disease_name_escaped = escape_latex(row['disease'])
    latex_string += f"{sig_num} & {disease_name_escaped} & {row['or_ci_str']} \\\\\n"
    current_sig = sig_num

latex_string += "\\bottomrule\n"
latex_string += "\\end{tabular}\n"
latex_string += "\\end{table}"

print("\\n--- Copy and Paste the LaTeX code below ---")
print(latex_string)

Processing checkpoint 1...
Processing checkpoint 2...
Processing checkpoint 3...
\nProcessing complete.
\n--- Copy and Paste the LaTeX code below ---
\begin{table}[h!]
\centering
\caption{Top 3 Most Consistent Diseases per Signature with Average Odds Ratio and 95\% CI}
\label{tab:signature_summary_ci}
\begin{tabular}{c p{9cm} c}
\toprule
\textbf{Signature} & \textbf{Top Consistently Associated Disease} & \textbf{Avg. OR (95\% CI)} \\
\midrule
0 & Atrial fibrillation and flutter & 4.97 (4.25 - 5.83) \\
0 & Heart failure NOS & 4.46 (4.09 - 4.88) \\
0 & Paroxysmal ventricular tachycardia & 3.62 (3.50 - 3.74) \\
\midrule
1 & Hallux valgus (Bunion) & 3.58 (3.44 - 3.73) \\
1 & Enthesopathy & 3.54 (3.33 - 3.76) \\
1 & Peripheral enthesopathies and allied syndromes & 3.46 (3.30 - 3.62) \\
\midrule
2 & Diaphragmatic hernia & 4.17 (3.89 - 4.48) \\
2 & Gastritis and duodenitis & 4.05 (3.82 - 4.30) \\
2 & Gastric ulcer & 3.71 (3.54 - 3.89) \\
\midrule
3 & Other tests & 3.20 (3.12 - 3.30) \\
3 & Ce